## Вариант деплоя номер 1

https://github.com/quant-aq/Plotly-Dash-Traefik-Example  
https://blog.quant-aq.com/deploying-web-based-analytic-dashboards-using-plotly-dash-docker-and-traefik/

## Вариант деплоя номер 2

Докер оперирует двумя терминами 
- контейнер 
это размороженный (развернутый) образ  
то есть мы берем образ и разворачиваем его в контейнер  
- образ  
это что-то настроенное и замороженной (операционная система, база данных)

Докер образ собирается из докер файла.   
Докер файл - это последовательность команд, которая собирает образ (например какую-то операционную систему с той начинкой, которая нам нужна)

Последовательность следующая  
- создается докер файл
- билдится докер файл и получается образ
- далее разворачивается образ в контейнер и можно работать

Образ для дэша можно создать на основае образа `tiangolo/uwsgi-nginx-flask`

У нас должна быть папка с приложениями (дашбордами)    
Какждая папка внутри этой папки это отдельное приложение (дашборд)  
И в этой же папке лежит `docker-compose.yml`  
Далее в какждой папке дашборда должен лежать свой `Dockerfile`  
Нам нужно в `docker-compose.yml` прописать в `services`    
наши апки (пути до них и как билдить)
далее мы можем находясь в папке с `docker-compose.yml`  
запускать отдельные дашборды или все вместе и управлять ими    

Важно файл с дашбордом называть  `main.py`  
и в этом фале должно быть (в отличии от обычного  запуска дэша)

In [ ]:
# В этом файле нужно прописать строчку для фласка вместе со строчкой для дэша
#  в сервер в даше прокинуть сервер фласка
# и конечно имопртировать flask  
# Инициализируем сервер
import flask
import dash
app = flask.Flask(__name__)
dash_app = dash.Dash(__name__, server = app, url_base_pathname = '/')
# Запускаем приложение так
if __name__ == '__main__':
    dash_app.run_server(host="0.0.0.0", debug=True, port=80)

Докер файл выглядит так

In [ ]:
# Dockerfile название без расширений
FROM tiangolo/uwsgi-nginx-flask:python3.8

COPY ./app /app

WORKDIR /app

RUN pip install --upgrade pip

RUN pip install -r requirements.txt

docker-compose выглядит так  
лежит в той же папке где докер файл

In [ ]:
# docker-compose.yml
version: '3'
services:
  web:
    # Ключевое слово "build" позволяет задать
    # путь к файлу Dockerfile, который нужно использовать для создания образа,
    # который позволит запустить сервис.
    build: ./app1
    volumes:
      - ./app:/app
    # Вспомните о том, что в качестве порта в 'server/server.py' указан порт 80.
    # Если мы хотим обратиться к серверу с нашего компьютера (находясь за пределами контейнера),
    # мы должны организовать перенаправление этого порта на порт компьютера.
    # Сделать это нам поможет ключевое слово 'ports'.
    # При его использовании применяется следующая конструкция: [порт компьютера]:[порт контейнера]
    # В нашем случае нужно использовать порт компьютера 80 и организовать его связь с портом
    # 80 контейнера (так как именно на этот порт сервер 
    # ожидает поступления запросов).      
    ports:
      - "80:80"
    # environment - настройки переменных окружения
    environment:
      - FLASK_APP=main.py
      - FLASK_DEBUG=1
      - 'RUN=flask run --host=0.0.0.0 --port=80'
    # 'command' - Команда, которую нужно запустить после создания образа.
    # command: flask run --host=0.0.0.0 --port=80
    # Infinite loop, to keep it alive, for debugging
    # command: bash -c "while true; do echo 'sleeping...' && sleep 10; done"

Сборка проекта

После того, как в docker-compose.yml внесены все необходимые инструкции, проект нужно собрать.   
Этот шаг нашей работы напоминает использование команды docker build, но соответствующая команда имеет отношение к нескольким сервисам:

In [ ]:
docker-compose build

Запуск проекта

Теперь, когда проект собран, пришло время его запустить. Этот шаг нашей работы соответствует шагу,   
на котором, при работе с отдельными контейнерами, выполняется команда docker run

In [ ]:
docker-compose up

Полезные команды

In [ ]:
# Эта команда позволяет останавливать и удалять контейнеры и другие ресурсы, созданные командой docker-compose up:
docker-compose down

In [ ]:
# Эта команда выводит журналы сервисов:
docker-compose logs -f [service name]

In [ ]:
# С помощью такой команды можно вывести список контейнеров:
docker-compose ps

In [ ]:
# Данная команда позволяет выполнить команду в выполняющемся контейнере:
docker-compose exec [service name] [command]

In [ ]:
# Такая команда позволяет вывести список образов:
docker-compose images